<a href="https://colab.research.google.com/github/dalexa10/FINDER_Summer_School_2023/blob/main/3_Program_analysis_methods/Hypothesis_testing_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hypothesis Tutorial for FinDeR Summer School

originally created by Mark Fuge, 2023

modified by Dario Rodriguez, 2023



This notebook provides some initial examples of how to use the [Hypothesis Library](https://hypothesis.readthedocs.io/) to do simple testing of functions, from simple fuzz testing to property based testing. We will start with simple examples and then extend in complexity as we go.



In [15]:
# Let's first install git to pull some files from github
!apt-get install git -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.9).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [35]:
#!pip install hypothesis
# Do the below instead if you want to test the optional auto-writing features
!pip install hypothesis[cli]

In [2]:
from operator import add
add(1,2) == 3
add(2,2) == 4
add(3,2) == 5

True

How would we check whether the addition is happening correctly? We could do a simple test by asserting it is similar to some reference solution:

In [3]:
def test_addition(a,b):
    assert add(a,b) == a+b

What kind of conditions would you run the above test against to check whether add was working properly?

Here is an example where there is a pretty bad and obvious bug in the code:

In [4]:
def bad_add(a,b):
    if (b>500):
        print(a,b)
        return -1
    else:
        print(a,b)
        return add(a,b)

Let's actually write this down the normal way like we would do in a typical test (see tests_normal.py)

In [17]:
# Now let's pull the first file from GitHub, it's called test_normal.py
!wget https://raw.githubusercontent.com/dalexa10/FINDER_Summer_School_2023/main/3_Program_analysis_methods/tests_normal.py

--2023-07-27 01:27:21--  https://raw.githubusercontent.com/dalexa10/FINDER_Summer_School_2023/main/3_Program_analysis_methods/tests_normal.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 526 [text/plain]
Saving to: ‘tests_normal.py’

tests_normal.py     100%[===================>]     526  --.-KB/s    in 0s      

2023-07-27 01:27:21 (19.1 MB/s) - ‘tests_normal.py’ saved [526/526]



In [18]:
!pytest tests_normal.py

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.2.2, pluggy-1.2.0
rootdir: /content
plugins: hypothesis-6.82.0, anyio-3.7.1
collected 2 items                                                              

tests_normal.py ..                                                       [100%]

============================== 2 passed in 0.01s ===============================


# Using Property Based Testing instead of manually writing the tests down

Here, we can use a library like Hypothesis to generate possible test cases.

In [19]:
from hypothesis import given
from hypothesis.strategies import integers

In [20]:
from hypothesis.strategies import floats
import numpy as np
from hypothesis.extra.numpy import arrays
# It can sample integers
print(integers().example())

# It can sample integers -- with some basic restrictions:
print(integers(10, 20).example())

# It can sample integers -- even with fairly complex restrictions
# (e.g., Even numbers)
print(integers().filter(lambda x: x % 2 == 0).example())

# Or even NumPy objects
print(arrays(dtype=np.float64,shape=(10,1)).example())
# You can put some limits on these
print(arrays(dtype=np.float64,shape=(10,1),
             elements=floats(1,200)).example())


-2613289582676000089
20
116
[[-6.83622027e+016]
 [ 1.90000000e+000]
 [-2.22507386e-313]
 [ 9.99990000e-001]
 [             inf]
 [ 6.10351562e-005]
 [-1.17549435e-038]
 [ 5.96046448e-008]
 [ 1.51299151e-217]
 [             nan]]
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [21]:
import hypothesis.strategies as st
# Uncomment to see other options for things it can generate:
print(dir(st))

['DataObject', 'DrawFn', 'SearchStrategy', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_internal', '_strategies', 'binary', 'booleans', 'builds', 'characters', 'complex_numbers', 'composite', 'data', 'dates', 'datetimes', 'decimals', 'deferred', 'dictionaries', 'emails', 'fixed_dictionaries', 'floats', 'fractions', 'from_regex', 'from_type', 'frozensets', 'functions', 'integers', 'ip_addresses', 'iterables', 'just', 'lists', 'none', 'nothing', 'one_of', 'permutations', 'random_module', 'randoms', 'recursive', 'register_type_strategy', 'runner', 'sampled_from', 'sets', 'shared', 'slices', 'text', 'timedeltas', 'times', 'timezone_keys', 'timezones', 'tuples', 'uuids']


Now, instead of manually specifying the test cases, you can actually just specify a range of inputs and it will test things randomly within this range:

In [22]:
@given(a=integers(), b=integers())
def test_addition(a,b):
    assert bad_add(a,b) == a+b


OK, now let's actually run the tests using something like pytest to actually get the failing examples (See tests_hypo.py), and let's print out what it is trying so that we can see what it is doing:

In [23]:
# Let's pull another file, this time is called test_hypo.py
!wget https://raw.githubusercontent.com/dalexa10/FINDER_Summer_School_2023/main/3_Program_analysis_methods/tests_hypo.py

--2023-07-27 01:29:09--  https://raw.githubusercontent.com/dalexa10/FINDER_Summer_School_2023/main/3_Program_analysis_methods/tests_hypo.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1216 (1.2K) [text/plain]
Saving to: ‘tests_hypo.py’

tests_hypo.py       100%[===================>]   1.19K  --.-KB/s    in 0s      

2023-07-27 01:29:09 (114 MB/s) - ‘tests_hypo.py’ saved [1216/1216]



In [24]:
!pytest tests_hypo.py

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.2.2, pluggy-1.2.0
rootdir: /content
plugins: hypothesis-6.82.0, anyio-3.7.1
collected 2 items                                                              

tests_hypo.py .F                                                         [100%]

=================================== FAILURES ===================================
______________________________ test_bad_addition _______________________________

    @given(a=integers(), b=integers())
>   def test_bad_addition(a,b):

tests_hypo.py:20: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

a = 0, b = -501

    @given(a=integers(), b=integers())
    def test_bad_addition(a,b):
>       assert bad_add(a,b) == a+b
E       assert -1 == (0 + -501)
E        +  where -1 = bad_add(0, -501)
E       Falsifying example: test_bad_addition(
E           a=0,
E           b=-501,
E       )

tests_hypo.py

Great! We seem to have found the error. We can attempt to fix the bug now, and to make sure that we capture it in the future, we can tell Hypothesis to directly look for this bug next time:

In [25]:
from hypothesis import example
@given(a=integers(), b=integers())
@example(0,201)
def test_bad_addition(a,b):
    assert bad_add(a,b) == a+b

In [26]:
!pytest tests_hypo.py

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.2.2, pluggy-1.2.0
rootdir: /content
plugins: hypothesis-6.82.0, anyio-3.7.1
collected 2 items                                                              

tests_hypo.py .F                                                         [100%]

=================================== FAILURES ===================================
______________________________ test_bad_addition _______________________________

    @given(a=integers(), b=integers())
>   def test_bad_addition(a,b):

tests_hypo.py:20: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

a = 0, b = -501

    @given(a=integers(), b=integers())
    def test_bad_addition(a,b):
>       assert bad_add(a,b) == a+b
E       assert -1 == (0 + -501)
E        +  where -1 = bad_add(0, -501)
E       Falsifying example: test_bad_addition(
E           a=0,
E           b=-501,
E       )

tests_hypo.py

How many of these tests should we do? We can specify this via the `settings` decorator. Too few and we may not catch the error. Too many and we may take forever!

In [27]:
from hypothesis import settings
@given(a=integers(), b=integers())
@settings(max_examples=10)
def test_bad_addition(a,b):
    assert bad_add(a,b) == a+b

OK, fine, but what if my function will never really be dealing with extreme values like the crazy integers that we see here. Can't I make this more helpful by restricting it to just the range of values that I care about? Absolutely! We can just change the testing strategy:

In [28]:
@given(a=integers(min_value=-500,max_value=500),
       b=integers(min_value=-500,max_value=500))
def test_bad_addition(a,b):
    assert bad_add(a,b) == a+b

We can also change the type of testing strategy that we use:

In [29]:
from hypothesis.strategies import floats
@given(a=floats(min_value=-500,max_value=500),
       b=integers(min_value=-500,max_value=500))
def test_bad_addition(a,b):
    assert bad_add(a,b) == a+b

# A (slightly) more complex example

We'll test a couple of functions to see if we can identify cases where they are not their own inverse (i.e., where $f(f(x)) = x$)

In [30]:
# Check whether a function is it's own inverse
def check_inverse(f,x):
    assert f(f(x)) == x

In [31]:
# Here we go, pulling yet another file
!wget https://raw.githubusercontent.com/dalexa10/FINDER_Summer_School_2023/main/3_Program_analysis_methods/test_inverse.py

--2023-07-27 01:36:04--  https://raw.githubusercontent.com/dalexa10/FINDER_Summer_School_2023/main/3_Program_analysis_methods/test_inverse.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 447 [text/plain]
Saving to: ‘test_inverse.py’

test_inverse.py     100%[===================>]     447  --.-KB/s    in 0s      

2023-07-27 01:36:04 (6.11 MB/s) - ‘test_inverse.py’ saved [447/447]



In [32]:
!pytest test_inverse.py

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.2.2, pluggy-1.2.0
rootdir: /content
plugins: hypothesis-6.82.0, anyio-3.7.1
collected 3 items                                                              

test_inverse.py .FF                                                      [100%]

=================================== FAILURES ===================================
___________________________________ test_1x ____________________________________

    @given(x = floats(allow_nan=False))
>   def test_1x(x):

test_inverse.py:13: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
test_inverse.py:15: in test_1x
    check_inverse(f,x)
test_inverse.py:5: in check_inverse
    assert f(f(x)) == x
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

a = 0.0

>   f = lambda a: 1/x
E   ZeroDivisionError: float division by zero
E   Falsifying example: test_1x(
E      

# A More Design/Engineering example: Detecting Logical Paradoxes

This example is taken directly from the [Hypothesis documentation](https://hypothesis.readthedocs.io/en/latest/examples.html#condorcet-s-paradox), wherein we can automatically discover a subtle paradox in social choice theory/voting called the [Condorcet Paradox](https://en.wikipedia.org/wiki/Condorcet_paradox). In this, we can find a case that breaks the formal property that a given voting system's preferences should be transitive. (See `condorcet_paradox.py`)

In [41]:
# Yet another .py file
!wget https://raw.githubusercontent.com/dalexa10/FINDER_Summer_School_2023/main/3_Program_analysis_methods/condorcet_paradox.py

--2023-07-27 01:57:04--  https://raw.githubusercontent.com/dalexa10/FINDER_Summer_School_2023/main/3_Program_analysis_methods/condorcet_paradox.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1189 (1.2K) [text/plain]
Saving to: ‘condorcet_paradox.py’

condorcet_paradox.p 100%[===================>]   1.16K  --.-KB/s    in 0s      

2023-07-27 01:57:04 (88.9 MB/s) - ‘condorcet_paradox.py’ saved [1189/1189]



In [42]:
!pytest condorcet_paradox.py

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.2.2, pluggy-1.2.0
rootdir: /content
plugins: hypothesis-6.82.0, anyio-3.7.1
collected 1 item                                                               

condorcet_paradox.py F                                                   [100%]

=================================== FAILURES ===================================
________________________ test_elections_are_transitive _________________________

    @given(lists(permutations(["A", "B", "C"]), min_size=3))
>   def test_elections_are_transitive(election):

condorcet_paradox.py:10: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

election = [['A', 'B', 'C'], ['B', 'C', 'A'], ['C', 'A', 'B']]

    @given(lists(permutations(["A", "B", "C"]), min_size=3))
    def test_elections_are_transitive(election):
        all_candidates = {"A", "B", "C"}
    
        # First calculate the pairwis

# Bonus: Writing Property Test code for us

For certain types of functions, if we appropriately type them, we can have it construct some of the testing code for us. This becomes more powerful for more complex datastructures, and is aided by the use of typing as we will see below.

(Note, this part requires the `!pip install hypothesis[cli]` at the beginning of the notebook.)

Let's say we have a simple subtraction function:

In [33]:
def subtract(a,b):
    return a-b

We can ask Hypothesis to generate some tests for us for this function:

In [36]:
# Last .py file pulled ...
!wget https://raw.githubusercontent.com/dalexa10/FINDER_Summer_School_2023/main/3_Program_analysis_methods/mark_math.py

--2023-07-27 01:56:08--  https://raw.githubusercontent.com/dalexa10/FINDER_Summer_School_2023/main/3_Program_analysis_methods/mark_math.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95 [text/plain]
Saving to: ‘mark_math.py’

mark_math.py        100%[===================>]      95  --.-KB/s    in 0s      

2023-07-27 01:56:08 (7.23 MB/s) - ‘mark_math.py’ saved [95/95]



In [37]:
!hypothesis write mark_math.subtract

# This test code was written by the `hypothesis.extra.ghostwriter` module
# and is provided under the Creative Commons Zero public domain dedication.

import mark_math
from hypothesis import given, strategies as st

# TODO: replace st.nothing() with appropriate strategies


@given(a=st.nothing(), b=st.nothing())
def test_fuzz_subtract(a, b):
    mark_math.subtract(a=a, b=b)



This is pretty disappointing, since it basically does nothing. Not very helpful! However, we didn't really give the library much information about the function. Here, we can use Python's typing system to provide some hints:

In [38]:
def subtract_wtypes(a: int, b: int):
    return a-b

In [39]:
!hypothesis write mark_math.subtract_wtypes > test_subtract.py

In [40]:
!pytest test_subtract.py

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.2.2, pluggy-1.2.0
rootdir: /content
plugins: hypothesis-6.82.0, anyio-3.7.1
collected 3 items                                                              

test_subtract.py FF.                                                     [100%]

=================================== FAILURES ===================================
______________ test_associative_binary_operation_subtract_wtypes _______________

    @given(
>       a=subtract_wtypes_operands, b=subtract_wtypes_operands, c=subtract_wtypes_operands
    )

test_subtract.py:11: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

a = 0, b = 0, c = 1

    @given(
        a=subtract_wtypes_operands, b=subtract_wtypes_operands, c=subtract_wtypes_operands
    )
    def test_associative_binary_operation_subtract_wtypes(a: int, b: int, c) -> None:
        left = mark_math.subtract_wtypes(a=a, 

OK! Now we see that because we specified that the inputs were typed as integers, it can figure out that this `subtract_wtypes` functions needs to (1) be commutative, (2) be associative, and (3) have an identity function.

Pretty cool. While this isn't terribly useful in this case, for more complex test cases this can rapidly help you cover anticipated test cases.